In [1]:
from scipy import spatial

def cosine(x, y):
    dataSetI = x
    dataSetII = y
    return 1 - spatial.distance.cosine(dataSetI, dataSetII)

In [13]:
import pickle

def load_category(category):
    pickle_in = open("elmo_category/"+ category +".pickle", "rb")
    return pickle.load(pickle_in)

def load_data(id_file):
    pickle_in = open("elmo_restaurant/"+ str(id_file) +".pickle", "rb")
    return pickle.load(pickle_in)[0]

In [14]:
import numpy as np

def similarity(category, word):
    res = []
    for cat in category:
        res.append(cosine(cat, word))
    return np.mean(res)

In [15]:
def sim_result(word):
    c1 = similarity(ambience, word)
    c2 = similarity(food, word)
    c3 = similarity(service, word)
    c4 = similarity(value, word)
    return c1, c2, c3, c4

In [16]:
def cat_str(a, b, c, d):
    if a > b and a > c and a > d:
        return 1
    elif b > a and b > c and b > d:
        return 2
    elif c > b and c > a and c > d:
        return 3
    elif d > b and d > c and d > a:
        return 4
    else:
        return 5

In [17]:
def str_cat(num):
    pred =''
    if num == 1:
        pred = 'AMBIENCE'
    elif num == 2:
        pred = 'FOOD'
    elif num == 3:
        pred = 'SERVICE'
    elif num == 4:
        pred = 'VALUE'
    return pred

In [18]:
ambience = load_category('ambience')
food = load_category('food')
service = load_category('service')
value = load_category('value_only_price')

In [19]:
import pandas as pd
from tqdm import tqdm_notebook

def run(tipe):
    df = pd.read_csv('Results/Aspect Terms Extraction/'+ tipe +'.csv')
    sf = pd.DataFrame(columns=['id', 'review', 'label', 'pred_multiple', 'terms'])
    for index in tqdm_notebook(range(len(df))):
        pred= ''
        pred_mult=[]
        if type(df['term'][index]) != float:
            terms = df['term'][index].split('|')
            id_file = df['id'][index]
            label = df['category'][index]
            review = df['review'][index]

            vector = load_data(id_file)
            tokenized = df['review'][index].split(' ')

            for term in terms:
                i = 0
                for token in tokenized:
                    try:
                        if term in token:
                            a, b, c, d = sim_result(vector[i])
                            x = cat_str(a, b, c, d)
                            pred_mult.append(str_cat(x))
                            break
                    except:
                        print(review,terms)
                        print(tokenized)
                    i += 1
        else:
            terms = ''
            id_file = df['id'][index]
            label = df['category'][index]
            review = df['review'][index]
            
        sf = sf.append({'id': id_file,
                        'review': review,
                        'label': label,
                        'pred_multiple': '|'.join(pred_mult),
                        'terms': '|'.join(terms)
                       }, ignore_index=True)

        sf.to_csv("Results/Categorization/"+ tipe +".csv")
        sf.to_excel("Results/Categorization/"+ tipe +".xlsx")

In [20]:
run('chunk')

In [21]:
run('20')